# Comprehensive Lab Setup with Context Management

This notebook demonstrates the complete MADSci lab setup process including:
- **Context Management**: Proper OwnershipInfo and MadsciContext usage
- **Resource Templates**: Leveraging templates for consistent resource creation
- **Modern Workflows**: New parameter system with input/feedforward separation
- **Internal Actions**: Built-in workflow control and validation

## Prerequisites

✅ All previous setup notebooks completed:
- 01_service_orchestration.ipynb
- 02_resource_templates.ipynb  
- 03_initial_resources.ipynb
- 04_validation.ipynb

## Setup and Context Initialization

In [ ]:
# Import all necessary modules
import json
import sys
from datetime import datetime
from pathlib import Path

try:
    from madsci.client.event_client import EventClient
    from madsci.client.experiment_client import ExperimentClient
    from madsci.client.resource_client import ResourceClient
    from madsci.client.workcell_client import WorkcellClient
    from madsci.common.types.auth_types import OwnershipInfo
    from madsci.common.types.context_types import MadsciContext
    from madsci.common.types.experiment_types import ExperimentDesign
    from madsci.common.utils import new_ulid_str

    print("✅ All MADSci modules imported successfully")

except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("Make sure MADSci is installed and services are running")
    sys.exit(1)

In [ ]:
# Initialize comprehensive MADSci context
madsci_context = MadsciContext(
    lab_server_url="http://localhost:8000",
    event_server_url="http://localhost:8001",
    experiment_server_url="http://localhost:8002",
    resource_server_url="http://localhost:8003",
    data_server_url="http://localhost:8004",
    workcell_server_url="http://localhost:8005",
)

print("🌐 MADSci Context Configuration:")
print("=" * 40)
for field, url in madsci_context.model_dump().items():
    if url:
        print(f"   {field}: {url}")

print("\n✅ Context configuration initialized")

In [ ]:
# Initialize all service clients with context
try:
    # Create clients using context URLs
    resource_client = ResourceClient(
        resource_server_url=str(madsci_context.resource_server_url)
    )
    workcell_client = WorkcellClient(
        workcell_server_url=str(madsci_context.workcell_server_url)
    )
    experiment_client = ExperimentClient(
        experiment_server_url=str(madsci_context.experiment_server_url)
    )
    event_client = EventClient(event_server_url=str(madsci_context.event_server_url))

    print("✅ All service clients initialized")

    # Test connectivity across all services
    connectivity_status = {}

    try:
        templates = resource_client.list_templates()
        connectivity_status["Resource Manager"] = (
            f"✅ {len(templates)} templates available"
        )
    except Exception as e:
        connectivity_status["Resource Manager"] = f"❌ {e}"

    try:
        workflows = workcell_client.list_workflows()
        connectivity_status["Workcell Manager"] = (
            f"✅ {len(workflows)} workflows available"
        )
    except Exception as e:
        connectivity_status["Workcell Manager"] = f"❌ {e}"

    # Display connectivity status
    print("\n🔌 Service Connectivity Status:")
    print("=" * 35)
    for service, status in connectivity_status.items():
        print(f"   {service}: {status}")

except Exception as e:
    print(f"❌ Client initialization failed: {e}")
    print("Check that all MADSci services are running")

## Create Lab Session with Full Context

In [ ]:
# Create comprehensive lab session context
session_id = new_ulid_str()
project_id = new_ulid_str()
campaign_id = new_ulid_str()

# Define lab session ownership
lab_ownership = OwnershipInfo(
    user_id="researcher_001",
    project_id=project_id,
    campaign_id=campaign_id,
    lab_id="example_lab_main",
    experiment_id=session_id,
)

print("🧪 Lab Session Context Created:")
print("=" * 35)
print(f"   Session ID: {session_id[:8]}...")
print(f"   Project ID: {project_id[:8]}...")
print(f"   Campaign ID: {campaign_id[:8]}...")
print(f"   Lab ID: {lab_ownership.lab_id}")
print(f"   User: {lab_ownership.user_id}")

# Create experiment design with context
experiment_design = ExperimentDesign(
    experiment_name=f"ComprehensiveDemo_{session_id[:8]}",
    experiment_description="Comprehensive demonstration of MADSci context management and modern features",
    owner=lab_ownership,
)

print("\n📋 Experiment Design:")
print(f"   Name: {experiment_design.experiment_name}")
print(f"   Description: {experiment_design.experiment_description}")
print("\n✅ Lab session context established")

## Resource Provisioning with Templates and Context

In [ ]:
# Create comprehensive resource set using templates with proper context

session_resources = {
    "plates": [],
    "tips": [],
    "reagents": [],
}

print("🔬 Provisioning Lab Resources with Context Management")
print("=" * 55)

# 1. Create experiment plates with ownership context
plate_configs = [
    {"name": "SourcePlate_001", "purpose": "sample_source", "priority": 1},
    {"name": "AssayPlate_001", "purpose": "primary_assay", "priority": 1},
    {"name": "AssayPlate_002", "purpose": "replicate_assay", "priority": 2},
    {"name": "WashPlate_001", "purpose": "wash_station", "priority": 3},
]

for config in plate_configs:
    try:
        plate = resource_client.create_resource_from_template(
            template_name="standard_96well_plate",
            resource_name=config["name"],
            overrides={
                "owner": lab_ownership.model_dump(),
                "attributes": {
                    "purpose": config["purpose"],
                    "priority": config["priority"],
                    "session_id": session_id,
                    "created_at": datetime.now().isoformat(),
                    "experiment_context": experiment_design.experiment_name,
                },
            },
            add_to_database=True,
        )

        session_resources["plates"].append({"resource": plate, "config": config})

        print(f"✅ Created {plate.resource_name} ({config['purpose']})")

    except Exception as e:
        print(f"❌ Failed to create {config['name']}: {e}")

print(f"\n📊 Created {len(session_resources['plates'])} plates with context tracking")

In [ ]:
# 2. Create tip racks with context
tip_configs = [
    {"name": "TipRack_LH1_001", "station": "liquid_handler_1", "position": 1},
    {"name": "TipRack_LH1_002", "station": "liquid_handler_1", "position": 2},
]

for config in tip_configs:
    try:
        tips = resource_client.create_resource_from_template(
            template_name="tip_rack_200ul",
            resource_name=config["name"],
            overrides={
                "owner": lab_ownership.model_dump(),
                "attributes": {
                    "assigned_station": config["station"],
                    "rack_position": config["position"],
                    "session_id": session_id,
                    "experiment_context": experiment_design.experiment_name,
                    "tip_count_remaining": 96,
                },
            },
            add_to_database=True,
        )

        session_resources["tips"].append({"resource": tips, "config": config})

        print(f"✅ Created {tips.resource_name} for {config['station']}")

    except Exception as e:
        print(f"❌ Failed to create {config['name']}: {e}")

print(
    f"\n📊 Created {len(session_resources['tips'])} tip racks with station assignments"
)

In [ ]:
# 3. Create reagents with full context tracking
reagent_configs = [
    {
        "name": "PBS_Buffer_Session",
        "type": "buffer",
        "volume": 100.0,
        "purpose": "washing_dilution",
    },
    {
        "name": "DMSO_Solvent_Session",
        "type": "solvent",
        "volume": 50.0,
        "purpose": "compound_dissolution",
    },
    {
        "name": "AssayBuffer_2X_Session",
        "type": "assay_buffer",
        "volume": 25.0,
        "purpose": "assay_reaction",
    },
]

for config in reagent_configs:
    try:
        reagent = resource_client.create_resource_from_template(
            template_name="generic_reagent",
            resource_name=config["name"],
            overrides={
                "owner": lab_ownership.model_dump(),
                "attributes": {
                    "reagent_type": config["type"],
                    "volume": config["volume"],
                    "remaining_volume": config["volume"],
                    "purpose": config["purpose"],
                    "session_id": session_id,
                    "experiment_context": experiment_design.experiment_name,
                    "lot_number": f"SES_{session_id[:8]}_{config['type'].upper()}",
                    "opened": False,
                    "expiry_date": "2025-12-31",
                },
            },
            add_to_database=True,
        )

        session_resources["reagents"].append({"resource": reagent, "config": config})

        volume = reagent.attributes.get("volume", "Unknown")
        purpose = reagent.attributes.get("purpose", "Unknown")
        print(f"✅ Created {reagent.resource_name} ({volume}mL, {purpose})")

    except Exception as e:
        print(f"❌ Failed to create {config['name']}: {e}")

print(
    f"\n📊 Created {len(session_resources['reagents'])} reagents with context tracking"
)

## Context-Aware Resource Inventory

In [ ]:
# Generate comprehensive resource inventory with context information
def generate_context_aware_inventory() -> None:
    """Generate detailed resource inventory with context tracking."""

    print("📦 Context-Aware Resource Inventory")
    print("=" * 40)
    print(f"Session: {session_id[:8]}... | Project: {project_id[:8]}...")
    print(f"Campaign: {campaign_id[:8]}... | Lab: {lab_ownership.lab_id}")
    print()

    total_resources = 0

    # Display plates with context
    print(f"🧪 PLATES ({len(session_resources['plates'])}):")
    for item in session_resources["plates"]:
        plate = item["resource"]
        config = item["config"]

        print(f"   • {plate.resource_name}")
        print(f"     ID: {plate.resource_id[:8]}...")
        print(f"     Purpose: {config['purpose']}")
        print(f"     Priority: {config['priority']}")
        print(f"     Owner: {plate.owner.user_id if plate.owner else 'Unassigned'}")
        print(
            f"     Experiment: {plate.owner.experiment_id[:8] if plate.owner and plate.owner.experiment_id else 'None'}..."
        )
        print()
        total_resources += 1

    # Display tips with context
    print(f"💧 TIP RACKS ({len(session_resources['tips'])}):")
    for item in session_resources["tips"]:
        tips = item["resource"]
        config = item["config"]

        print(f"   • {tips.resource_name}")
        print(f"     ID: {tips.resource_id[:8]}...")
        print(f"     Station: {config['station']}")
        print(f"     Position: {config['position']}")
        remaining = tips.attributes.get("tip_count_remaining", "Unknown")
        print(f"     Tips remaining: {remaining}")
        print()
        total_resources += 1

    # Display reagents with context
    print(f"🧪 REAGENTS ({len(session_resources['reagents'])}):")
    for item in session_resources["reagents"]:
        reagent = item["resource"]
        config = item["config"]

        print(f"   • {reagent.resource_name}")
        print(f"     ID: {reagent.resource_id[:8]}...")
        print(f"     Type: {config['type']}")
        print(f"     Purpose: {config['purpose']}")
        volume = reagent.attributes.get("volume", "Unknown")
        remaining = reagent.attributes.get("remaining_volume", "Unknown")
        print(f"     Volume: {remaining}/{volume}mL")
        lot = reagent.attributes.get("lot_number", "Unknown")
        print(f"     Lot: {lot}")
        print()
        total_resources += 1

    print(f"📊 Total Resources: {total_resources}")
    print(f"🔗 All resources linked to session: {session_id[:8]}...")


# Generate the inventory
generate_context_aware_inventory()

## Modern Workflow Execution with New Parameter System

In [ ]:
# Prepare workflow execution with modern parameter system

# Select resources for workflow
primary_plate = session_resources["plates"][0]["resource"]  # Source plate
assay_plate = session_resources["plates"][1]["resource"]  # Assay plate
buffer_reagent = session_resources["reagents"][0]["resource"]  # PBS buffer

print("⚙️  Preparing Modern Workflow Execution")
print("=" * 40)
print("Selected Resources:")
print(
    f"   Source Plate: {primary_plate.resource_name} ({primary_plate.resource_id[:8]}...)"
)
print(f"   Assay Plate: {assay_plate.resource_name} ({assay_plate.resource_id[:8]}...)")
print(
    f"   Buffer: {buffer_reagent.resource_name} ({buffer_reagent.resource_id[:8]}...)"
)

# Prepare workflow inputs using new parameter system
workflow_inputs = {
    # Input parameters (provided at submission)
    "json_inputs": {
        "experiment_id": lab_ownership.experiment_id,
        "user_id": lab_ownership.user_id,
        "plate_id": primary_plate.resource_id,
        "reagent_buffer_id": buffer_reagent.resource_id,
        "context_demo": "true",
    },
    # File inputs
    "file_inputs": {"protocol_file": "./protocols/enhanced_demo_protocol.py"},
    # Context information for workflow execution
    "context": {
        "ownership": lab_ownership.model_dump(),
        "experiment_design": experiment_design.model_dump(),
        "session_metadata": {
            "session_id": session_id,
            "created_at": datetime.now().isoformat(),
            "resource_count": sum(
                len(resources) for resources in session_resources.values()
            ),
        },
    },
}

print("\n📋 Workflow Input Parameters:")
print(f"   JSON inputs: {len(workflow_inputs['json_inputs'])} parameters")
print(f"   File inputs: {len(workflow_inputs['file_inputs'])} files")
print("   Context data: Full ownership and session tracking")
print("\n✅ Workflow parameters prepared")

In [ ]:
# Create enhanced protocol file for demonstration
protocol_content = '''#!/usr/bin/env python3
"""
Enhanced Demo Protocol - Context-Aware Liquid Handling

This protocol demonstrates context-aware operations with proper
ownership tracking and modern MADSci features.
"""

def main(context=None):
    """Main protocol execution with context awareness."""

    print("🧪 Starting Enhanced Demo Protocol")

    if context:
        print(f"   Experiment: {context.get('experiment_id', 'Unknown')}")
        print(f"   User: {context.get('user_id', 'Unknown')}")
        print(f"   Session: {context.get('session_id', 'Unknown')}")

    # Simulate protocol steps
    steps = [
        "Initialize liquid handler",
        "Load tips and plates",
        "Prepare buffer dilutions",
        "Transfer samples",
        "Execute mixing protocol",
        "Generate results file"
    ]

    for i, step in enumerate(steps, 1):
        print(f"   Step {i}: {step}")

    print("✅ Protocol completed successfully")

    return {
        "status": "completed",
        "steps_executed": len(steps),
        "context_tracked": context is not None
    }

if __name__ == "__main__":
    main()
'''

# Create protocols directory and write protocol
protocol_dir = Path("../protocols")
protocol_dir.mkdir(exist_ok=True)

protocol_file = protocol_dir / "enhanced_demo_protocol.py"
protocol_file.write_text(protocol_content)

print(f"✅ Created enhanced protocol: {protocol_file}")
print("   Features: Context awareness, ownership tracking, modern structure")

## Execute Enhanced Workflow with Modern Features

In [ ]:
# Execute the enhanced workflow with full context management
print("🚀 Executing Enhanced Context Workflow")
print("=" * 40)

try:
    # Submit workflow with all modern features
    workflow_run = workcell_client.submit_workflow(
        "./workflows/enhanced_context_workflow.workflow.yaml", **workflow_inputs
    )

    print("✅ Workflow submitted successfully")
    print(f"   Run ID: {workflow_run.workflow_run_id[:8]}...")
    print(f"   Status: {workflow_run.status}")
    print("   Context: Fully tracked with ownership info")

    # Log workflow initiation with context
    event_client.info(
        "Enhanced workflow initiated",
        extra={
            "workflow_run_id": workflow_run.workflow_run_id,
            "ownership": lab_ownership.model_dump(),
            "session_id": session_id,
            "resource_count": sum(
                len(resources) for resources in session_resources.values()
            ),
            "workflow_features": [
                "input_parameter_separation",
                "feedforward_parameters",
                "file_based_parameters",
                "context_management",
                "internal_actions",
            ],
        },
    )

    workflow_run_id = workflow_run.workflow_run_id

except Exception as e:
    print(f"❌ Workflow submission failed: {e}")
    workflow_run_id = None

print("\n📋 Modern Workflow Features Demonstrated:")
features = [
    "✅ Input Parameter Separation (json_inputs, file_inputs)",
    "✅ Feed-Forward Parameters (step-to-step data flow)",
    "✅ File-Based Parameter Handling",
    "✅ Context Management (OwnershipInfo, MadsciContext)",
    "✅ Internal Workcell Actions (validate_step, generate_report)",
    "✅ Enhanced Error Handling (retries, timeouts)",
    "✅ Resource Template Integration",
]

for feature in features:
    print(f"   {feature}")

## Monitor Workflow with Context Tracking

In [ ]:
# Monitor workflow execution with comprehensive context tracking
import time

if workflow_run_id:
    print(f"📊 Monitoring Workflow: {workflow_run_id[:8]}...")
    print("=" * 45)

    try:
        # Poll workflow status
        monitoring_count = 0
        max_monitoring = 30  # Prevent infinite loop in demo

        while monitoring_count < max_monitoring:
            status = workcell_client.get_workflow_run_status(workflow_run_id)

            print(f"\n⏱️  Monitor Check {monitoring_count + 1}:")
            print(f"   Status: {status.status}")
            print(f"   Steps: {len(status.steps) if status.steps else 0}")

            # Show step details with context
            if status.steps:
                for step in status.steps[-3:]:  # Show last 3 steps
                    step_status = step.status if hasattr(step, "status") else "Unknown"
                    step_name = (
                        step.step_name if hasattr(step, "step_name") else "Unknown"
                    )
                    print(f"     • {step_name}: {step_status}")

            # Log monitoring event with context
            event_client.info(
                f"Workflow monitoring check {monitoring_count + 1}",
                extra={
                    "workflow_run_id": workflow_run_id,
                    "status": str(status.status),
                    "ownership": lab_ownership.model_dump(),
                    "session_id": session_id,
                },
            )

            # Check if workflow is complete
            if status.status in ["completed", "failed", "cancelled"]:
                print(f"\n🏁 Workflow finished with status: {status.status}")
                break

            monitoring_count += 1
            time.sleep(3)  # Wait before next check

        if monitoring_count >= max_monitoring:
            print("\n⏰ Monitoring limit reached - workflow may still be running")

    except Exception as e:
        print(f"❌ Workflow monitoring failed: {e}")

else:
    print("❌ No workflow to monitor - submission failed")

## Session Summary and Context Export

In [ ]:
# Generate comprehensive session summary with full context information
from typing import Any, Dict


def generate_session_summary() -> Dict[str, Any]:
    """Create detailed summary of the lab session with context tracking."""

    return {
        "session_info": {
            "session_id": session_id,
            "project_id": project_id,
            "campaign_id": campaign_id,
            "created_at": datetime.now().isoformat(),
            "lab_id": lab_ownership.lab_id,
            "user_id": lab_ownership.user_id,
        },
        "experiment_design": experiment_design.model_dump(),
        "madsci_context": madsci_context.model_dump(),
        "resources_created": {
            "plates": [
                {
                    "name": item["resource"].resource_name,
                    "id": item["resource"].resource_id,
                    "purpose": item["config"]["purpose"],
                    "template_used": "standard_96well_plate",
                }
                for item in session_resources["plates"]
            ],
            "tips": [
                {
                    "name": item["resource"].resource_name,
                    "id": item["resource"].resource_id,
                    "station": item["config"]["station"],
                    "template_used": "tip_rack_200ul",
                }
                for item in session_resources["tips"]
            ],
            "reagents": [
                {
                    "name": item["resource"].resource_name,
                    "id": item["resource"].resource_id,
                    "type": item["config"]["type"],
                    "volume": item["config"]["volume"],
                    "template_used": "generic_reagent",
                }
                for item in session_resources["reagents"]
            ],
        },
        "workflow_execution": {
            "workflow_file": "enhanced_context_workflow.workflow.yaml",
            "run_id": workflow_run_id if "workflow_run_id" in locals() else None,
            "parameter_types": {
                "json_inputs": list(workflow_inputs["json_inputs"].keys()),
                "file_inputs": list(workflow_inputs["file_inputs"].keys()),
                "context_provided": True,
            },
        },
        "features_demonstrated": [
            "Resource Templates with Context",
            "OwnershipInfo Propagation",
            "MadsciContext Configuration",
            "Modern Workflow Parameters",
            "Input/Feed-Forward Separation",
            "File-Based Parameters",
            "Internal Workcell Actions",
            "Enhanced Error Handling",
            "Context-Aware Monitoring",
        ],
    }


# Generate and display summary
session_summary = generate_session_summary()

print("🎉 Session Summary - Enhanced MADSci Features Demo")
print("=" * 55)

print("\n📋 Session Information:")
print(f"   Session ID: {session_summary['session_info']['session_id'][:8]}...")
print(f"   Project ID: {session_summary['session_info']['project_id'][:8]}...")
print(f"   Lab: {session_summary['session_info']['lab_id']}")
print(f"   User: {session_summary['session_info']['user_id']}")

print("\n🔬 Resources Created:")
for category, resources in session_summary["resources_created"].items():
    print(f"   {category.capitalize()}: {len(resources)} items")
    for resource in resources:
        print(f"     • {resource['name']} (Template: {resource['template_used']})")

print("\n⚙️  Features Demonstrated:")
for feature in session_summary["features_demonstrated"]:
    print(f"   ✅ {feature}")

# Export session summary
summary_file = Path("../session_summaries") / f"session_{session_id[:8]}.json"
summary_file.parent.mkdir(exist_ok=True)

summary_file.write_text(json.dumps(session_summary, indent=2, default=str))

print(f"\n💾 Session summary exported to: {summary_file}")
print("   Use this file for session tracking and audit purposes")

## Best Practices Summary

🎯 **Context Management Best Practices:**

### OwnershipInfo Usage
- ✅ **Always set ownership** for resources and workflows
- ✅ **Use consistent IDs** across experiment lifecycle
- ✅ **Track hierarchical ownership** (project → campaign → experiment)
- ✅ **Propagate context** through all operations

### MadsciContext Configuration
- ✅ **Centralize service URLs** in MadsciContext
- ✅ **Use environment-specific contexts** (dev/staging/prod)
- ✅ **Validate connectivity** before operations
- ✅ **Share context** across clients

### Modern Workflow Parameters
- ✅ **Separate input types** (json_inputs, file_inputs, feed_forward)
- ✅ **Use feed-forward** for step-to-step data flow
- ✅ **Leverage internal actions** for workflow control
- ✅ **Include context data** in workflow submissions

### Resource Template Integration
- ✅ **Use templates consistently** for resource creation
- ✅ **Set ownership context** in all template instantiations
- ✅ **Include session metadata** in resource attributes
- ✅ **Track resource lifecycle** with proper context

---

**🚀 Next Steps:**

You have successfully completed the comprehensive lab setup with all modern MADSci features! This notebook demonstrated:

1. **Full Context Integration** - Proper ownership and context tracking
2. **Resource Template Usage** - Efficient resource provisioning
3. **Modern Parameter System** - Input/feedforward separation
4. **Internal Actions** - Advanced workflow control
5. **Enhanced Error Handling** - Robust execution patterns

**Continue to:** Advanced scenarios in `tutorials/` directory for complex multi-step workflows and specialized use cases.